In [2]:
import sys
import os
import pandas as pd
import numpy as np
import optuna
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

# 1) Aseguramos que la raíz del proyecto esté en el path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

c:\Users\leona\anaconda3\envs\py310lab3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2) Carga de datos
sell_in       = pd.read_csv('../data/sell-in.txt', sep='\t')
prod_vigentes = pd.read_csv('../data/product_id_apredecir201912.txt', sep='\t')
productos     = pd.read_csv('../data/tb_productos_05262025.txt', sep='\t')

# 3) Parseo de 'periodo' a datetime (añadimos día ‘01’)
sell_in['periodo'] = pd.to_datetime(
    sell_in['periodo'].astype(str) + '01',
    format='%Y%m%d'
)

# 4) Agregación mensual por product_id
sell_in_agg = (
    sell_in
    .groupby(['periodo','product_id'], as_index=False)
    .agg({'tn':'sum','cust_request_qty':'sum','cust_request_tn':'sum'})
)
# Filtrar sólo productos vigentes
sell_in_agg = sell_in_agg.merge(
    prod_vigentes[['product_id']],
    on='product_id',
    how='inner'
).drop(columns=['cust_request_qty','cust_request_tn'])

# 5) Crear columna objetivo tn_mas_2 (shift -2)
sell_in_agg['tn_mas_2'] = sell_in_agg.groupby('product_id')['tn'].shift(-2)

# 6) Generar features de lag y rolling
# 6.1) media móvil últimos 3 meses
sell_in_agg['mean_last_3m'] = (
    sell_in_agg
    .groupby('product_id')['tn']
    .apply(lambda x: x.rolling(window=3, min_periods=1).mean())
    .reset_index(level=0, drop=True)
)
# 6.2) lags 1 a 11
for lag in range(1, 12):
    sell_in_agg[f'tn_{lag}'] = sell_in_agg.groupby('product_id')['tn'].shift(lag)
# Renombrar tn a tn_0
sell_in_agg.rename(columns={'tn':'tn_0'}, inplace=True)

# 7) Definir lista de columnas de entrada
feature_cols = [f'tn_{i}' for i in range(12)]  # ['tn_0','tn_1',…,'tn_11']

# 8) Construir dataset de entrenamiento: todo hasta 2018-12-01
cutoff = pd.to_datetime('2018-12-01')
dataset_training = (
    sell_in_agg[sell_in_agg['periodo'] <= cutoff]
    .dropna(subset=['tn_mas_2'] + feature_cols)
)

# 9) Filtrar sólo los “productos mágicos” si lo deseas
magicos = [
 20002,20003,20006,20010,20011,20018,20019,20021,
 20026,20028,20035,20039,20042,20044,20045,20046,
 20049,20051,20052,20053,20055,20008,20001,20017,
 20086,20180,20193,20320,20532,20612,20637,20807,20838
]
df_magicos = dataset_training[dataset_training['product_id'].isin(magicos)]

In [4]:
df_magicos

,periodo,product_id,tn_0,tn_mas_2,mean_last_3m,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11
5714,2017-12-01,20001,1049.38860,1043.76470,1356.539413,1580.47401,1439.75563,1316.94604,1267.39462,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222
5715,2017-12-01,20002,820.58984,712.00087,809.487020,750.41853,857.45269,1065.34529,619.71078,845.39319,968.15756,843.43785,522.35365,834.73521,505.88633,550.15707
5716,2017-12-01,20003,961.66344,788.30749,1448.833020,1426.23717,1958.59845,1465.83347,864.21928,785.12398,744.08829,620.48202,525.82591,917.16548,752.11520,1063.45835
5719,2017-12-01,20006,530.36517,588.56272,707.223273,724.31168,866.99297,599.26956,781.41139,670.77751,698.31833,813.17643,565.31925,868.34182,599.18590,528.40996
5721,2017-12-01,20008,543.14221,506.25385,632.731910,631.86060,723.19292,671.24800,837.31906,563.83449,739.24267,849.48973,626.51768,937.42393,631.71430,523.04609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13288,2018-12-01,20532,5.89124,6.87043,6.011617,6.30858,5.83503,4.52676,8.37937,5.62638,6.07581,7.41620,7.08716,12.24797,9.29433,10.03272
13335,2018-12-01,20612,2.80400,4.30324,3.433480,3.25956,4.23688,3.83447,3.92728,3.77255,4.89148,4.59071,4.65267,5.49298,3.83887,4.19274
13345,2018-12-01,20637,4.92167,4.66338,4.647197,4.28775,4.73217,4.81740,3.98088,4.78519,3.26566,5.63963,6.40351,6.66175,5.58230,4.89165
13428,2018-12-01,20807,1.90008,2.36726,1.815773,1.86118,1.68606,1.96850,1.29391,1.57927,2.01622,2.20976,2.47203,2.98221,2.46268,2.23571


In [5]:
df_magicos.shape

(422, 16)

In [6]:
feature_cols

['tn_0',
 'tn_1',
 'tn_2',
 'tn_3',
 'tn_4',
 'tn_5',
 'tn_6',
 'tn_7',
 'tn_8',
 'tn_9',
 'tn_10',
 'tn_11']

In [7]:
X_train = df_magicos[feature_cols]
# X_train tiene que tener product_id como índice pero tomado desde df_magicos
X_train.index = df_magicos['product_id']
y_train = df_magicos['tn_mas_2']

# 10) Función objetivo para Optuna (maximizar R² con TimeSeriesSplit)
def objective(trial):
    params = {
        'n_estimators':        trial.suggest_int('n_estimators', 50, 500),
        'max_depth':           trial.suggest_int('max_depth', 3, 12),
        'learning_rate':       trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves':          trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples':   trial.suggest_int('min_child_samples', 5, 50),
        'subsample':           trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree':    trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'random_state':        42,
        'verbosity':           -1
    }
    model = LGBMRegressor(**params)
    tscv = TimeSeriesSplit(n_splits=10)
    scores = cross_val_score(
        model, X_train, y_train,
        cv=tscv,
        scoring='r2',
        n_jobs=-1
    )
    return scores.mean()

# 11) Lanzar optimización
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=600)

print("Mejores hiperparámetros:", study.best_params)

# 12) Entrenar modelo final con los mejores parámetros
best_params = study.best_params

[I 2025-07-16 20:17:31,534] A new study created in memory with name: no-name-d8046a7c-46f5-46c7-8bd5-5775046a159b
C:\Users\leona\AppData\Local\Temp\ipykernel_35864\3746631295.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':       trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
C:\Users\leona\AppData\Local\Temp\ipykernel_35864\3746631295.py:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample':           trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\leona\AppData\Local\Temp\ipykernel_35864\3746631295.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/

Mejores hiperparámetros: {'n_estimators': 363, 'max_depth': 9, 'learning_rate': 0.009875622909936115, 'num_leaves': 66, 'min_child_samples': 9, 'subsample': 0.8012950699729687, 'colsample_bytree': 0.5380074345629984}


In [8]:
X_train.shape, y_train.shape

((422, 12), (422,))

In [9]:
X_train

,tn_0,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11
product_id,,,,,,,,,,,,
20001,1049.38860,1580.47401,1439.75563,1316.94604,1267.39462,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222
20002,820.58984,750.41853,857.45269,1065.34529,619.71078,845.39319,968.15756,843.43785,522.35365,834.73521,505.88633,550.15707
20003,961.66344,1426.23717,1958.59845,1465.83347,864.21928,785.12398,744.08829,620.48202,525.82591,917.16548,752.11520,1063.45835
20006,530.36517,724.31168,866.99297,599.26956,781.41139,670.77751,698.31833,813.17643,565.31925,868.34182,599.18590,528.40996
20008,543.14221,631.86060,723.19292,671.24800,837.31906,563.83449,739.24267,849.48973,626.51768,937.42393,631.71430,523.04609
...,...,...,...,...,...,...,...,...,...,...,...,...
20532,5.89124,6.30858,5.83503,4.52676,8.37937,5.62638,6.07581,7.41620,7.08716,12.24797,9.29433,10.03272
20612,2.80400,3.25956,4.23688,3.83447,3.92728,3.77255,4.89148,4.59071,4.65267,5.49298,3.83887,4.19274
20637,4.92167,4.28775,4.73217,4.81740,3.98088,4.78519,3.26566,5.63963,6.40351,6.66175,5.58230,4.89165


In [10]:
best_model = LGBMRegressor(**best_params, random_state=42, verbosity=-1)
best_model.fit(X_train, y_train)
print("R² en entrenamiento:", best_model.score(X_train, y_train))

R² en entrenamiento: 0.9721771180279025


In [11]:
dataset_201912_vigentes = (
    sell_in_agg[sell_in_agg['periodo'] == '2019-12-01']
      .merge(prod_vigentes[['product_id']], on='product_id', how='inner')
)
dataset_201912_vigentes, dataset_201912_vigentes.shape

(       periodo  product_id        tn_0  tn_mas_2  mean_last_3m        tn_1  \
 0   2019-12-01       20001  1504.68856       NaN   1487.855463  1397.37231   
 1   2019-12-01       20002  1087.30855       NaN   1496.807430  1423.57739   
 2   2019-12-01       20003   892.50129       NaN    974.053890   948.29393   
 3   2019-12-01       20004   637.90002       NaN    808.846137   723.94206   
 4   2019-12-01       20005   593.24443       NaN    732.312970   606.91173   
 ..         ...         ...         ...       ...           ...         ...   
 775 2019-12-01       21263     0.01270       NaN      0.020230     0.03247   
 776 2019-12-01       21265     0.05007       NaN      0.075093     0.06600   
 777 2019-12-01       21266     0.05121       NaN      0.078883     0.06713   
 778 2019-12-01       21267     0.01569       NaN      0.050990     0.04052   
 779 2019-12-01       21276     0.00892       NaN      0.021040     0.03341   
 
            tn_2        tn_3        tn_4        tn

In [12]:
dataset_201912_vigentes.drop(columns=['tn_mas_2'], inplace=True)

In [13]:
dataset_656 = dataset_201912_vigentes.dropna()
dataset_656, dataset_656.shape

(       periodo  product_id        tn_0  mean_last_3m        tn_1        tn_2  \
 0   2019-12-01       20001  1504.68856   1487.855463  1397.37231  1561.50552   
 1   2019-12-01       20002  1087.30855   1496.807430  1423.57739  1979.53635   
 2   2019-12-01       20003   892.50129    974.053890   948.29393  1081.36645   
 3   2019-12-01       20004   637.90002    808.846137   723.94206  1064.69633   
 4   2019-12-01       20005   593.24443    732.312970   606.91173   996.78275   
 ..         ...         ...         ...           ...         ...         ...   
 770 2019-12-01       21248     0.01129      0.017877     0.02964     0.01270   
 772 2019-12-01       21256     0.01271      0.016000     0.02682     0.00847   
 773 2019-12-01       21259     0.01412      0.021173     0.02965     0.01975   
 774 2019-12-01       21262     0.01834      0.020697     0.02682     0.01693   
 775 2019-12-01       21263     0.01270      0.020230     0.03247     0.01552   
 
            tn_3        tn

In [14]:
dataset_124 = dataset_201912_vigentes[dataset_201912_vigentes.isna().any(axis=1)]
dataset_124, dataset_124.shape

(       periodo  product_id       tn_0  mean_last_3m       tn_1       tn_2  \
 31  2019-12-01       20032  527.79811    728.554807  906.69823  751.16808   
 111 2019-12-01       20127  170.32792    273.648603  463.80054  186.81735   
 144 2019-12-01       20174   91.11780    102.493790   68.84756  147.51601   
 171 2019-12-01       20210   58.78261    106.976320  134.38972  127.75663   
 174 2019-12-01       20213   58.84043     87.258370  104.19056   98.74412   
 ..         ...         ...        ...           ...        ...        ...   
 771 2019-12-01       21252    0.08560      0.101890    0.11982    0.10025   
 776 2019-12-01       21265    0.05007      0.075093    0.06600    0.10921   
 777 2019-12-01       21266    0.05121      0.078883    0.06713    0.11831   
 778 2019-12-01       21267    0.01569      0.050990    0.04052    0.09676   
 779 2019-12-01       21276    0.00892      0.021040    0.03341    0.02079   
 
           tn_3       tn_4       tn_5       tn_6       tn_7   

In [15]:
lags = [f"tn_{i}" for i in range(1, 12)]

# ejemplo de DataFrame
# df = pd.read_csv(… )

# 1) Calcula la media por fila (ignorando NaN) en las columnas tn_1…tn_11
row_means = dataset_124[lags].mean(axis=1)

# 2) Rellena los NaN de cada columna con la media de su propia fila
dataset_124[lags] = dataset_124[lags].apply(
    lambda row: row.fillna(row.mean()),
    axis=1
)

# —– o equivalentemente —–

dataset_124[lags] = dataset_124[lags].fillna(row_means, axis=0)

C:\Users\leona\AppData\Local\Temp\ipykernel_35864\353778797.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_124[lags] = dataset_124[lags].apply(
C:\Users\leona\AppData\Local\Temp\ipykernel_35864\353778797.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_124[lags] = dataset_124[lags].fillna(row_means, axis=0)


In [16]:
dataset_124, dataset_124.shape

(       periodo  product_id       tn_0  mean_last_3m       tn_1       tn_2  \
 31  2019-12-01       20032  527.79811    728.554807  906.69823  751.16808   
 111 2019-12-01       20127  170.32792    273.648603  463.80054  186.81735   
 144 2019-12-01       20174   91.11780    102.493790   68.84756  147.51601   
 171 2019-12-01       20210   58.78261    106.976320  134.38972  127.75663   
 174 2019-12-01       20213   58.84043     87.258370  104.19056   98.74412   
 ..         ...         ...        ...           ...        ...        ...   
 771 2019-12-01       21252    0.08560      0.101890    0.11982    0.10025   
 776 2019-12-01       21265    0.05007      0.075093    0.06600    0.10921   
 777 2019-12-01       21266    0.05121      0.078883    0.06713    0.11831   
 778 2019-12-01       21267    0.01569      0.050990    0.04052    0.09676   
 779 2019-12-01       21276    0.00892      0.021040    0.03341    0.02079   
 
           tn_3        tn_4        tn_5        tn_6        tn_

In [17]:
# Merge de dataset_124 con dataset_656
dataset_final = pd.concat([dataset_124, dataset_656], ignore_index=True)
dataset_final.drop(columns=['mean_last_3m','periodo'], inplace=True)
# dataset_final que queden 12 columnas y que product_id sea el índice
dataset_final.set_index('product_id', inplace=True)


In [18]:
dataset_final.shape

(780, 12)

In [19]:
dataset_final

,tn_0,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11
product_id,,,,,,,,,,,,
20032,527.79811,906.69823,751.16808,629.90072,703.140590,698.026270,605.549310,488.434710,361.382200,573.573240,247.998800,596.587215
20127,170.32792,463.80054,186.81735,12.80399,221.140627,221.140627,221.140627,221.140627,221.140627,221.140627,221.140627,221.140627
20174,91.11780,68.84756,147.51601,129.99999,120.971360,171.231580,73.998010,118.760752,118.760752,118.760752,118.760752,118.760752
20210,58.78261,134.38972,127.75663,7.78032,89.975557,89.975557,89.975557,89.975557,89.975557,89.975557,89.975557,89.975557
20213,58.84043,104.19056,98.74412,86.10789,42.718040,82.940153,82.940153,82.940153,82.940153,82.940153,82.940153,82.940153
...,...,...,...,...,...,...,...,...,...,...,...,...
21248,0.01129,0.02964,0.01270,0.01411,0.021170,0.021160,0.009880,0.015530,0.031060,0.053650,0.062090,0.029620
21256,0.01271,0.02682,0.00847,0.00423,0.029650,0.028220,0.009880,0.015530,0.018350,0.059300,0.050810,0.038110
21259,0.01412,0.02965,0.01975,0.00564,0.031060,0.046570,0.009880,0.019760,0.021170,0.067770,0.050800,0.042340


In [ ]:
# 15) Realizar predicción
X_new = dataset_final
y_pred = best_model.predict(X_new)

predicciones = pd.DataFrame(
    data=y_pred,
    index=X_new.index,
    columns=['tn']
)

predicciones = predicciones.sort_values('product_id',ascending=True ).reset_index()


In [27]:
predicciones

,product_id,tn
0,20001,1422.830836
1,20002,1426.250284
2,20003,706.897729
3,20004,509.104973
4,20005,510.210541
...,...,...
775,21263,10.176211
776,21265,10.176211
777,21266,10.176211
778,21267,10.176211


In [28]:
predicciones.sort_values('product_id', inplace=True)
predicciones.to_csv('../data/predicciones_lgbm_v2.csv', index=False)

print("Predicciones guardadas en ../data/predicciones_lgbm_v2.csv")

Predicciones guardadas en ../data/predicciones_lgbm_v2.csv
